# Experiment-2: Logistic Regression and SVM
## In this experiment, we will: 
- (1) Get more intuitions about Logistic Regression and SVM. 
- (2) Get your hands dirty on [a9a dataset](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#a9a), which is one of the [LIBSVM Dataset](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/).
- (3) Get some experience about hyper-parameter tuning, loss function selection in SVM, optimizer selection, initilizer selection.
### As usual, in this experiment, we will use the mini framework [simple_ml](https://github.com/lizhaoliu-Lec/simple_ml), which means simple machine learning, written by [lizhaoliu-Lec](https://github.com/lizhaoliu-Lec/).

In [2]:
# as usual, do a little setup
%pip install simple_ml
import numpy as np

from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

from simple_ml.utils.distance import euclidean
from simple_ml.preprocessing.general import Standardizer
from simple_ml.nn.layer import Input, Linear, Dropout, Softmax
from simple_ml.nn.model import Model
from simple_ml.nn.initializer import zeros, ones
from simple_ml.nn.regularizer import L2_Regularizer, L1_Regularizer, L1L2_Regularizer
from simple_ml.nn.optimizer import SGD, Momentum, Adam, RMSProp
from simple_ml.utils.metric import accuracy, absolute_error, square_error

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Load the (preprocessed) a9a data.
# fix random seed
np.random.seed(1234)

# some utils func
def convert_to_onehot(x):
    return np.array(x == 1, dtype=np.float)

# read the data
def get_a9a_data(data_path='./tmp/exp2/a9a'):
    X_train, y_train = load_svmlight_file(data_path)
    X_test, y_test = load_svmlight_file(data_path + '.t', n_features=123)
    X_train, X_test = X_train.A, X_test.A
    y_train, y_test = np.reshape(y_train, (-1, 1)), np.reshape(y_test, (-1, 1))

    return X_train, X_test, y_train, y_test
    

X_train, X_test, y_train, y_test = get_a9a_data()
print('We have `%d` training examples and `%d` test examples' % (X_train.shape[0], X_test.shape[0]))
print('X_train shape: ', X_train.shape, ', X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape, ', y_test shape: ', y_test.shape)
print('But we set aside 20% of the training examples for validation.\n')
val_split = 0.2
val_size = int(X_train.shape[0] * val_split)
X_val, y_val, X_train, y_train = X_train[:val_size], y_train[:val_size], X_train[val_size:], y_train[val_size:]
print('Finally, we have...')
print('We have `%d` training examples, `%d` validation examples and `%d` test examples' % (X_train.shape[0], X_val.shape[0],X_test.shape[0]))
print('X_train shape: ', X_train.shape, ', X_val shape: ', X_val.shape, ', X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape, ', y_val shape: ', y_val.shape, ', y_test shape: ', y_test.shape)

We have `32561` training examples and `16281` test examples
X_train shape:  (32561, 123) , X_test shape:  (16281, 123)
y_train shape:  (32561, 1) , y_test shape:  (16281, 1)
But we set aside 20% of the training examples for validation.

Finally, we have...
We have `26049` training examples, `6512` validation examples and `16281` test examples
X_train shape:  (26049, 123) , X_val shape:  (6512, 123) , X_test shape:  (16281, 123)
y_train shape:  (26049, 1) , y_val shape:  (6512, 1) , y_test shape:  (16281, 1)


## Normalized the data:
As usual, we normalize the data.

In [5]:
from simple_ml.preprocessing import Standardizer

print('Before normalized the data...')
print('*** X_train *** \nmean: \n%s \nvariance: \n%s' % (np.mean(X_train, axis=0), np.var(X_train, axis=0)))
print('*** X_val *** \nmean: \n%s \nvariance: \n%s' % (np.mean(X_val, axis=0), np.var(X_val, axis=0)))
print('*** X_test *** \nmean: \n%s \nvariance: \n%s' % (np.mean(X_test, axis=0), np.var(X_test, axis=0)))
standardizer = Standardizer()
standardizer.fit(X_train)
X_train = standardizer.transform(X_train)
X_val = standardizer.transform(X_val)
X_test = standardizer.transform(X_test)

print('\nAfter normalized the data...')
print('*** X_train *** \nmean: \n%s \nvariance: \n%s' % (np.mean(X_train, axis=0), np.var(X_train, axis=0)))
print('*** X_val *** \nmean: \n%s \nvariance: \n%s' % (np.mean(X_val, axis=0), np.var(X_val, axis=0)))
print('*** X_test *** \nmean: \n%s \nvariance: \n%s' % (np.mean(X_test, axis=0), np.var(X_test, axis=0)))

Before normalized the data...
*** X_train *** 
mean: 
[1.96514262e-01 1.80467580e-01 2.11601213e-01 1.94709970e-01
 2.16706975e-01 6.98337748e-01 7.77764981e-02 3.35905409e-02
 2.98667895e-02 6.43402818e-02 4.01550923e-02 4.99059465e-04
 2.30335138e-04 2.00122845e-01 2.01428078e-01 1.99086337e-01
 2.02042305e-01 1.97320435e-01 1.64804791e-01 2.22810856e-01
 3.57787247e-02 3.22392414e-01 1.79661407e-02 3.29763139e-02
 4.23432761e-02 1.54708434e-02 1.99239894e-02 1.38584974e-02
 5.29003033e-02 5.22092979e-03 2.83696111e-02 1.30523245e-02
 1.04802488e-02 1.65073515e-03 1.30753580e-01 3.22392414e-01
 2.22810856e-01 7.53195900e-02 2.48723559e-01 4.60670275e-01
 1.36550347e-01 3.27344620e-01 3.15175247e-02 3.05961841e-02
 1.26300434e-02 6.91005413e-04 2.86767246e-02 1.25993320e-01
 1.01501017e-01 1.10445698e-01 1.25225536e-01 1.27720834e-01
 4.29958924e-02 6.15378709e-02 1.15781796e-01 3.08265193e-02
 4.86774924e-02 4.76025951e-03 2.01543245e-02 2.68724327e-04
 4.75642059e-02 1.55130715e-01 

## Logistic Regression
We first recap the math of Logistic Regression.
Given $$ \textbf{X} \in \mathbb{R}^{m \times n} $$, where m is the number of examples, and n is the number of features;

And learnable parameters $$ \textbf{w} \in \mathbb{R}^{n \times h}$$, $$ \textbf{b} \in \mathbb{R}^{h}$$.

Linear Model is $$ \textbf{y} = \textbf{X} \cdot \textbf{w} + \textbf{b} $$.

We can add a column of ones in $$ \textbf{X} $$, so it become fully Vectorized form, because we concate $$ \textbf{w} $$ and $$ \textbf{b} $$ together. 

So we get $$ \textbf{X} \in \mathbb{R}^{m \times (n + 1)}$$ and learnable parameter $$ \textbf{W} \in \mathbb{R}^{(n + 1) \times h}$$

Then we make the derivation of Closed Form Solution.
For Linear Regression problem, we want to minimize the loss function $$ J(\hat{y}, y) = \frac{1}{2m}\Sigma_{i=1}^{m}(\hat{y}_{i} - y_{i})  $$

where $$ \hat{y} =  \textbf{X} \cdot \textbf{W} \in \mathbb{R}^{m \times h}$$ is prediction output by the Linear Model.

We can write it as Vectorized form as \begin{eqnarray} J(\hat{y}, y) &=& \frac{1}{2m}(\hat{y} - y)^{T} \cdot (\hat{y} - y) \\
&=& \frac{1}{2m}(\textbf{X} \textbf{W} - y)^{T} \cdot (\textbf{X} \textbf{W} - y) \\
\textbf{W}^{*} &=& \mathop{\arg\min}_{W} J(\hat{y}, y) \end{eqnarray} 

Set $$ \frac{\partial{J}}{\partial{W}} = 0 $$, we get $$ \textbf{W}^{*} = (\textbf{X}^{T}\textbf{X})^{-1}\textbf{X}^{T}\textbf{y} $$
Note that $$ (\textbf{X}^{T}\textbf{X}) $$ maybe uninvertible, so in practice, we use pesudo inverse or Ridge Regression.

Now, it's time to get your hands dirty!

In [ ]:
# we first build the linear regression model.

# build the linear model
print(10 * '*' + ' LinearRegression model begin ' + 10 * '*')
linear_regression = LinearRegression()
# get closed form solution
linear_regression.fit(X_train, y_train)
test_y_hat = linear_regression.predict(X_test)
train_y_hat = linear_regression.predict(X_train)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]
print('Training mean square error: ', training_error)
print('Test mean square error: ', test_error)
print(10 * '*' + ' LinearRegression model end ' + 10 * '*')
print()

## Ridge Regression
Ridge Regression model can be seen as a regularized Linear Model.

And it doesn't have any un invertible problem when getting closed form solution.

The closed form solution of ridge regression is computed as following.
$$ \textbf{W}^{*} = (\textbf{X}^{T}\textbf{X} + \lambda \mathbb{I})^{-1}\textbf{X}^{T}\textbf{y} $$

In [ ]:
# build the ridge regression model
print(10 * '*' + ' Ridge LinearRegression model begin ' + 10 * '*')
# the lambda is represented as alpha in the API, by default, alpha=1
ridge_regression = RidgeRegression(alpha=1)
# get closed form solution
ridge_regression.fit(X_train, y_train)
test_y_hat = ridge_regression.predict(X_test)
train_y_hat = ridge_regression.predict(X_train)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]
print('Training mean square error: ', training_error)
print('Test mean square error: ', test_error)
print(10 * '*' + ' Ridge LinearRegression model end ' + 10 * '*')
print()

## Different lambdas in Ridge Regression
Now we explore the different alpha's impact in Ridge Regression.

We explore it in some magnitude, from 1e-7 to 1e3.

We can see that, when the lambda is small, e.g., from 1e-7 to 10, the both training error and validation error is very small.
But when lambda become larger than 10, both error become very large.

In [ ]:
from tqdm import tqdm

magnitude_range = [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3]
lambdas = [1e-7,1e-6,1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]
training_errors = []
validation_errors = []
for l in tqdm(lambdas):
    ridge_regression = RidgeRegression(alpha=l)
    # get closed form solution
    ridge_regression.fit(X_train, y_train)
    val_y_hat = ridge_regression.predict(X_val)
    train_y_hat = ridge_regression.predict(X_train)
    training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
    val_error = square_error(y_val, val_y_hat) / y_val.shape[0]
    training_errors.append(training_error)
    validation_errors.append(val_error)

    
plt.plot(magnitude_range, np.array(training_errors), label='training-error')
plt.plot(magnitude_range, np.array(validation_errors), label='validation-error')
plt.xlabel('lambda magnitude')
plt.ylabel('error')
plt.xticks(magnitude_range)
plt.legend()
plt.show()
print("In validation set, the min error occurs when lambda is ", lambdas[np.argmin(validation_errors)], '.')

## Test best lambda's performance.
Now we test the best lambda's performance on test set. Note that all the hyper-parameter tuning must perform on validation set or training set.
And we test the final result on the test set.

In [ ]:
print(10 * '*' + ' Best Ridge LinearRegression model begin ' + 10 * '*')
# the lambda is represented as alpha in the API, by default, alpha=1
ridge_regression = RidgeRegression(alpha=1e-7)
# get closed form solution
ridge_regression.fit(X_train, y_train)
test_y_hat = ridge_regression.predict(X_test)
train_y_hat = ridge_regression.predict(X_train)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]
print('Training mean square error: ', training_error)
print('Test mean square error: ', test_error)
print(10 * '*' + ' Best Ridge LinearRegression model end ' + 10 * '*')
print()

# Linear Regression: Optimize it using Gradient Descent

Recap: 

We have Linear model: $$ \hat{y} =  \textbf{X} \cdot \textbf{W} $$, where $$ \hat{y} \in \mathbb{R}^{m \times h}, \textbf{X} \in \mathbb{R}^{m \times (n+1)}, \mathbb{W}^{(n+1) \times h} $$

We have Loss function (Least Square Loss): 
\begin{eqnarray} J(\hat{y}, y) &=& \frac{1}{2m}(\hat{y} - y)^{T} \cdot (\hat{y} - y) \\
&=& \frac{1}{2m}(\textbf{X} \textbf{W} - y)^{T} \cdot (\textbf{X} \textbf{W} - y) \end{eqnarray} 

The Gradient w.r.t $$\textbf{W}$$ is:
 
 \begin{eqnarray} \frac{\partial J}{W} &=& \frac{1}{2m} \frac{\partial J}{\hat{y}} \frac{\partial \hat{y}}{W} \\
 &=&  \frac{1}{m} X^{T}(\textbf{X} \textbf{W} - y)   \end{eqnarray}
 
With learning rate $$\eta \in \mathbb{R} $$, which typically is a very small float number, e.g., 1e-3, 1e-4...
 
Gradient Descent Update the $$\textbf{W}$$ using the following equation.

$$ \textbf{W}_{t+1} = \textbf{W}_t - \eta \frac{\partial J}{W} $$ 

But all of the details are warp as A Layer for you to call in simple_ml.

Note that, In Gradient descent, we use all the training data to update the parameter in each run.

We can see that, using the whole data to update typically need more iterations to fully converge, comparing with Stochastic
Gradient Descent(SGD) or Mini Batch Gradient Descent(MBGD).
But GD fully use the vectorization computation, which make it super fast, you can see that, the whole process uses less 
than 1 seconds.

In [ ]:
from datetime import datetime
print(10 * '*' + ' GD Linear model ' + 10 * '*')

# build the linear model with gradient descent
# define layer
Inputs = Input(input_shape=X_train.shape[1])
linear_out = Linear(output_dim=1, activation=None, initializer=ones)(Inputs)
model = Model(Inputs, linear_out)
model.compile('MSE', optimizer=SGD(lr=0.01))
begin = datetime.now()
model.fit(X_train, y_train,
          verbose=100, epochs=500,
          validation_data=(X_val, y_val),
          batch_size=X_train.shape[0], # here we set batch_size 
                                       # equal to number of training examples 
                                       # to implement Gradient Descent.
                                
          metric='MAE',
          shuffle=False,
          # peek_type='single-reg'
          )
end = datetime.now()
plt.subplot(211)
plt.plot(model.train_losses, label='train_losses')
plt.plot(model.validation_losses, label='valid_losses')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.subplot(212)
plt.plot(model.train_metrics, label='train_metrics')
plt.plot(model.validation_metrics, label='valid_metrics')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

train_y_hat = model.forward(X_train)
test_y_hat = model.forward(X_test)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]

print('Training error: ', training_error)
print('Test error: ', test_error)
print('Cost: ',  (end - begin), ' seconds')
print(10 * '*' + ' GD Linear model end ' + 10 * '*')
print()

# Linear Regression: Optimize it using Stochastic Gradient Descent(SGD)

Note that, In Stochastic Gradient descent, we use one training example to update the parameter in each run.

We can see that, SGD brings fast convergence compared with GD.
But SGD hasn't use the vectorization computation at all, which make it super slow, you can see that, the whole process uses 25x 
times more than GD.

In [ ]:
print(10 * '*' + ' SGD Linear model ' + 10 * '*')

# build the linear model with gradient descent
# define layer
Inputs = Input(input_shape=X_train.shape[1])
linear_out = Linear(output_dim=1, activation=None, initializer=ones)(Inputs)
model = Model(Inputs, linear_out)
model.compile('MSE', optimizer=SGD(lr=0.01))
begin = datetime.now()
model.fit(X_train, y_train,
          verbose=100, epochs=500,
          validation_data=(X_val, y_val),
          batch_size=1, # here we set batch_size = 1 
                        # to implement Stochastic Gradient Descent.
                                
          metric='MAE',
          shuffle=False,
          # peek_type='single-reg'
          )
end = datetime.now()
plt.subplot(211)
plt.plot(model.train_losses, label='train_losses')
plt.plot(model.validation_losses, label='valid_losses')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.subplot(212)
plt.plot(model.train_metrics, label='train_metrics')
plt.plot(model.validation_metrics, label='valid_metrics')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

train_y_hat = model.forward(X_train)
test_y_hat = model.forward(X_test)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]

print('Training error: ', training_error)
print('Test error: ', test_error)
print('Cost: ',  (end - begin), ' seconds')
print(10 * '*' + ' SGD Linear model end ' + 10 * '*')
print()

# Linear Regression: Optimize it using Mini Batch Gradient Descent(MBGD)

In the above comparision between GD and SGD, we can clearly see that:

|      Metric     |       GD      |      SGD      |
| --------------- | ------------- | ------------- |
| Time-Efficiency |       Yse     |       No      |
| Convergence     |      Slow     |      Fast     |

Clearly there is a trade off to be made. But how can we balance the trade off ?

The key is to control the batch_size.
- GD: batch_size: all examples
- SGD: batch_size: 1
- MBGD: batch_size: between 1 and all examples, typically 16, 32, 64...

Now we compare the influence w.r.t time efficiency and convergence.

As stated before, GD and MBGD both have its metric and drawback, what we can do is making trade off.
As illustrated by the following three graphs, the smaller the batch_size, the faster for convergence,
but cost more time. So in practice, we set batch_size bigger than one, but smaller than the number of 
examples, typically 32, 64, 128...

In [ ]:
batch_sizes = [1, 8, 16, 32, 64, 128, X_train.shape[0]]
batch_size2training_losses = {b: None for b in batch_sizes}
batch_size2validation_losses = {b: None for b in batch_sizes}
batch_size2timecost = []

for b in batch_sizes:
    print('Starting with batch_size = %d' % b)
    Inputs = Input(input_shape=X_train.shape[1])
    linear_out = Linear(output_dim=1, activation=None, initializer=ones)(Inputs)
    model = Model(Inputs, linear_out)
    model.compile('MSE', optimizer=SGD(lr=0.01))
    begin = datetime.now()
    model.fit(X_train, y_train,
              verbose=100, epochs=500,
              validation_data=(X_val, y_val),
              batch_size=b, # here we set batch_size with a bunch of sizes 
                            # to implement both SGD, MBGD and GD               
              metric='MAE',
              shuffle=False,
              # peek_type='single-reg'
              )
    end = datetime.now()
    cost = (end - begin).seconds
    batch_size2training_losses[b] = model.train_losses
    batch_size2validation_losses[b] = model.valid_losses
    batch_size2timecost.append(cost)
    print('End with batch_size = %d\n' % b)

for b in batch_sizes:
    plt.plot(batch_size2training_losses[b], label='batch_size = %d' % b)
plt.xlabel('epoch')
plt.ylabel('$MSE_{train}$')
plt.legend()
plt.savefig('batch_size_train.png', dpi=300)
plt.show()

for b in batch_sizes:
    plt.plot(batch_size2validation_losses[b], label='batch_size = %d' % b)
plt.xlabel('epoch')
plt.ylabel('$MSE_{val}$')
plt.legend()
plt.savefig('batch_size_val.png', dpi=300)
plt.show()

plt.plot(batch_sizes, batch_size2timecost, marker='v')
plt.xlabel('batch_size')
plt.ylabel('time cost (Seconds)')
plt.grid()
plt.savefig('time_cost.png', dpi=300)
plt.show()

# Tuning Hyper-Parameter
Now it's time to tuning the hyper-parameter of Gradient Descent Algorithm.
In the simple GD, the only hyper-parameter is learning rate $$\eta$$.

Now we validate the best $$ \eta $$ in validation set and test our final model.
We first search it in log space, e.g., 1e-4, 1e-3, 1e-2, 1e-1, 1.
As Bigger learning rate will lead to very big loss, we do not try it here.
As seen below, the best learning rate is 1e-2.

The bigger the learning rate, the faster the loss decreases, but it may not converge to a very low loss.

The smaller the learning rate, it may converge to a very low loss, but the loss decrease very slowly. 

In [ ]:
learning_rates = [1e-4, 1e-3, 1e-2, 1e-1, 1]
training_losses = []
validation_losses = []

for lr in learning_rates:
    print('Starting with learning_rate = %.4f' % lr)
    Inputs = Input(input_shape=X_train.shape[1])
    linear_out = Linear(output_dim=1, activation=None, initializer=ones)(Inputs)
    model = Model(Inputs, linear_out)
    model.compile('MSE', optimizer=SGD(lr=lr))
    model.fit(X_train, y_train,
              verbose=200, epochs=200, # we set epochs to 1000 for fully convergence
              validation_data=(X_val, y_val),
              batch_size=64, # here we set batch_size to 64 considering the trade off            
              metric='MAE',
              shuffle=False,
              # peek_type='single-reg'
              )
    training_losses.append(model.training_losses)
    validation_losses.append(model.valid_losses)
    print('End with learning_rate = %.4f\n' % lr)

for lr, train_loss in zip(learning_rates, training_losses):
    plt.plot(train_loss, label='lr = %1.0e' % lr)
plt.xlabel('epoch')
plt.ylabel('$MSE_{train}$')
plt.legend(loc='upper right', prop = {'size':7})
plt.savefig('lr_train.png', dpi=300)
plt.show()

for lr, val_loss in zip(learning_rates, validation_losses):
    plt.plot(val_loss, label='lr = %1.0e' % lr)
plt.xlabel('epoch')
plt.ylabel('$MSE_{val}$')
plt.legend(loc='upper right', prop = {'size':8})
plt.savefig('lr_val.png', dpi=300)
plt.show()

## The Difference of $$\textbf{W}$$
Now we explore the differences of $$\textbf{W}$$ in Closed Form Solution, Ridge Regression and MBGD.

More specifically, we will evaluate the $$l_2norm$$ of $$\textbf{W}$$ under different setting.

Given $$ \textbf{x} \in \mathbb{R}^{n} $$, $$ l_2norm(\textbf{x}) = \sqrt{\Sigma_{i=1}^{n}\textbf{x}_{i}^{2}}$$


In [ ]:
magnitude_range = [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3]
lambdas = [1e-7,1e-6,1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

linear_Ws = len(lambdas) * [np.linalg.norm(linear_regression.beta[1:], ord=2, axis=1)[0]]
ridge_Ws = []

for l in tqdm(lambdas):
    ridge_regression = RidgeRegression(alpha=l)
    # get closed form solution
    ridge_regression.fit(X_train, y_train)
    ridge_Ws.append(np.linalg.norm(ridge_regression.beta[1:], ord=2, axis=1)[0])

    
plt.plot(magnitude_range, np.array(linear_Ws), label='linear regression', marker='v')
plt.plot(magnitude_range, np.array(ridge_Ws), label='ridge regression', marker='v')
plt.xlabel('lambda magnitude')
plt.ylabel('$W$ magnitude')
plt.xticks(magnitude_range)
plt.legend()
plt.grid()
plt.savefig('ridge_w.png', dpi=300)
plt.show()


In [ ]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

NUM_EPOCH = 500

linear_Ws = NUM_EPOCH * [np.linalg.norm(linear_regression.beta[1:], ord=2, axis=1)[0]]

MBGD_Ws = []
Inputs = Input(input_shape=X_train.shape[1])
linear_out = Linear(output_dim=1, activation=None)(Inputs)
MBGD_linear_regression = Model(Inputs, linear_out)

for epoch in range(NUM_EPOCH):
    MBGD_linear_regression.compile('MSE', optimizer=SGD(lr=0.01)) # use the best lambda here.
    MBGD_linear_regression.fit(X_train, y_train,
              verbose=-1, epochs=1, # we set epochs to 1000 for fully convergence
              validation_data=(X_val, y_val),
              batch_size=64, # here we set batch_size to 64 considering the trade off            
              metric='MAE',
              shuffle=False)
    MBGD_Ws.append(np.linalg.norm(linear_out.weight.T, ord=2, axis=1)[0])
    
plt.plot(linear_Ws, label='CFS')
plt.plot(MBGD_Ws, label='MBGD')
plt.xlabel('epoch')
plt.ylabel('$W$ magnitude')
plt.legend()
plt.grid()
plt.legend( loc='upper right')
plt.savefig('mbgd_w.png', dpi=300)
plt.show()

## The Performance
In this section, we report the performance on CFS optimized LR, RR and MBGD optimized LR in training set, validation set and test set. 
The $\lambda$ is set to 1e-7 after validation. And batch size is set to 64 in MBGD, learing rate set to 0.01 and epoch is set to 200 by validation.



In [ ]:
print(10 * '*' + ' Best LinearRegression model ' + 10 * '*')
# the lambda is represented as alpha in the API, by default, alpha=1
linear_regression = LinearRegression()
# get closed form solution
linear_regression.fit(X_train, y_train)
test_y_hat = linear_regression.predict(X_test)
val_y_hat = linear_regression.predict(X_val)
train_y_hat = linear_regression.predict(X_train)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
val_error = square_error(y_val, val_y_hat) / y_val.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]
print('Training mean square error: ', training_error)
print('Val mean square error: ', val_error)
print('Test mean square error: ', test_error)
print(10 * '*' + ' Best LinearRegression model end ' + 10 * '*')
print()

print(10 * '*' + ' Best Ridge LinearRegression model ' + 10 * '*')
# the lambda is represented as alpha in the API, by default, alpha=1
ridge_regression = RidgeRegression(alpha=1e-7)
# get closed form solution
ridge_regression.fit(X_train, y_train)
test_y_hat = ridge_regression.predict(X_test)
val_y_hat = ridge_regression.predict(X_val)
train_y_hat = ridge_regression.predict(X_train)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
val_error = square_error(y_val, val_y_hat) / y_val.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]
print('Training mean square error: ', training_error)
print('Val mean square error: ', val_error)
print('Test mean square error: ', test_error)
print(10 * '*' + ' Best Ridge LinearRegression model end ' + 10 * '*')
print()

print(10 * '*' + ' Best MBGD Linear model ' + 10 * '*')
# build the linear model with gradient descent
# define layer
Inputs = Input(input_shape=X_train.shape[1])
linear_out = Linear(output_dim=1, activation=None, initializer=ones)(Inputs)
model = Model(Inputs, linear_out)
model.compile('MSE', optimizer=SGD(lr=0.01))
model.fit(X_train, y_train,
          verbose=-1, epochs=200,
          validation_data=(X_val, y_val),
          batch_size=64,             
          metric='MAE',
          shuffle=False,
          # peek_type='single-reg'
          )
train_y_hat = model.forward(X_train)
test_y_hat = model.forward(X_test)
val_y_hat = model.forward(X_val)
training_error = square_error(y_train, train_y_hat) / y_train.shape[0]
test_error = square_error(y_test, test_y_hat) / y_test.shape[0]
val_error = square_error(y_val, val_y_hat) / y_val.shape[0]

print('Training error: ', training_error)
print('Val mean square error: ', val_error)
print('Test error: ', test_error)
print(10 * '*' + ' Best MBGD Linear model end ' + 10 * '*')
print()

